In [1]:
import os 
import json
import cv2
import numpy as np
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from coreLib.utils import *
%matplotlib inline
tqdm.pandas()

label_path="/media/ansary/DriveData/Work/bengalAI/datasets/words"
labels=os.listdir(label_path)
labels

['আসবো',
 'হিসাবে',
 'বসন্ত',
 'মৃত্যুশয্যার',
 'সমদ্র',
 'সা্বাধীনতার',
 'উৎপন্ন',
 'বেইমানী',
 'ডুবুরীরা',
 'বিজয়',
 'কারবার',
 'ঘরে',
 'অগ্রতি',
 'করেও',
 'নেক',
 'লাগল',
 'অবিকল্প',
 'ধানমন্ডি',
 'সামগ্রীর',
 'শাশুড়ির',
 'অর্থনীতির',
 'খন্ড',
 'কলেজ',
 'বোঝার',
 'গ্রন্থ',
 'তলিয়ে',
 'ভাতা',
 'ফাক্ক',
 'সুপ্তির',
 'উচ্চ',
 'ঝগড়া',
 'করের',
 'বাজায়',
 'স্থায়ী',
 'আজ',
 'কেজি',
 'পরাজয়',
 'আন্দোলনে',
 'এক্ষেত্রে',
 'মানুষ্যত্ব',
 'হচ্ছে',
 'দুুর্জন',
 'তিরমিযী',
 'অপেক্ষায়',
 'ঘুড়ি',
 'দড়ি',
 'নিউইয়র্ক',
 'শাসনামলে',
 'বাং',
 'লাফাইয়া',
 'হয়েছিল',
 'চাচ্ছি',
 'পালিয়ে',
 'কোটি',
 'শিখিয়ে',
 'আগামীতে',
 'সকালে',
 'তাইতে',
 'দেখার',
 'নিয়ম',
 'জার্মানি',
 'রশ্মি',
 'ঢেলা',
 'ইত্যাদির',
 'শমাশীল',
 'চিকিৎসার',
 'খাতাটা',
 'বহুমাত্রিক',
 'দৈনন্দি',
 'ছাপানো',
 'ছারা',
 'সন্তানের',
 'সূত্রাবলি',
 'দেখলে',
 'আদিম',
 'সলিলের',
 'ব্যাখ্য',
 'আলয়ে',
 'ফুটতো',
 'গাছের',
 'উপস্থাপন',
 'অফিসার',
 'বর্জ্য',
 'মুশকিল',
 'করেছিল',
 'বাংলার',
 'যাতে',
 'পূজা',
 'অবিচা',
 'কেটে',
 'উপরই',
 'বীজগনিতিক',

In [2]:

# class map
class_map_csv=os.path.join(os.getcwd(),"resources","class_map_corrected.csv")
# labels for graphemes
grapheme_labels_csv=os.path.join(os.getcwd(),"resources","label.csv")
# read class map
df_map=pd.read_csv(class_map_csv)
# get grapheme roots
df_root = df_map.groupby('component_type').get_group('grapheme_root')
df_root.index = df_root['label']
df_root = df_root.drop(columns = ['label','component_type'])
# get vowel_diacritic
df_vd = df_map.groupby('component_type').get_group('vowel_diacritic')
df_vd.index = df_vd['label']
df_vd = df_vd.drop(columns = ['label','component_type'])
# get consonant_diacritic
df_cd = df_map.groupby('component_type').get_group('consonant_diacritic')
df_cd.index = df_cd['label']
df_cd = df_cd.drop(columns = ['label','component_type'])
# get grapheme labels
df_grapheme=pd.read_csv(grapheme_labels_csv)
# filter columns
df_grapheme=df_grapheme[['image_id','grapheme']]
df_grapheme

,image_id,grapheme
0,Train_23740,র্ন্ত
1,Train_101878,স্ত
2,Train_5736,ল্ডো
3,Train_76463,গা
4,Train_158052,দু
...,...,...
200835,Train_129198,ফ্র্যা
200836,Train_175699,মা
200837,Train_38444,ঙ্গ
200838,Train_27077,দ্ভ


In [3]:
def word2grapheme(word):
    '''
        creates a grapheme list for a given word
        args:
            word  : the word to find grapheme for
        returns:
            list of graphemes
    '''
    graphemes = []
    grapheme = ''
    i = 0
    # iterate over the word
    while i < len(word):    
        grapheme+=(word[i])
        # pass for '্' 
        if word[i] in ['\u200d','্']:
            pass 
        # special case for 'ঁ'
        elif  word[i] == 'ঁ':
            graphemes.append(grapheme)
            grapheme = ''
        # if char in root    
        elif word[i] in df_root.values:
            if i+1 ==len(word):
                graphemes.append(grapheme)
            elif word[i+1] not in ['্', '\u200d', 'ঁ'] + list(df_vd.values):
                graphemes.append(grapheme)
                grapheme = ''

        elif word[i] in df_vd.values:
            if i+1 ==len(word):
                graphemes.append(grapheme)
            elif word[i+1] != 'ঁ':
                graphemes.append(grapheme)
                grapheme = ''                

        i = i+1

    return graphemes

In [4]:

df=pd.DataFrame({"words":labels})
df

,words
0,আসবো
1,হিসাবে
2,বসন্ত
3,মৃত্যুশয্যার
4,সমদ্র
...,...
5079,বাহিনী
5080,শয়তান
5081,মাইক্রো
5082,দেবীর


In [5]:
df['graphemes']=df['words'].progress_apply(lambda x: word2grapheme(x))
df

  0%|          | 0/5084 [00:00<?, ?it/s]

,words,graphemes
0,আসবো,"[আ, স, বো]"
1,হিসাবে,"[হি, সা, বে]"
2,বসন্ত,"[ব, স, ন্ত]"
3,মৃত্যুশয্যার,"[মৃ, ত্যু, শ, য্যা, র]"
4,সমদ্র,"[স, ম, দ্র]"
...,...,...
5079,বাহিনী,"[বা, হি, নী]"
5080,শয়তান,"[শ, য়, তা, ন]"
5081,মাইক্রো,"[মা, ই, ক্রো]"
5082,দেবীর,"[দে, বী, র]"


In [6]:
grapheme_list=[]
for glist in tqdm(df.graphemes.tolist()):
    grapheme_list+=glist
print(len(grapheme_list))
grapheme_list=list(set(grapheme_list))
len(grapheme_list)

  0%|          | 0/5084 [00:00<?, ?it/s]

17129


755

In [7]:
non_found=[]
for g in tqdm(grapheme_list):
    tdf=df_grapheme.loc[df_grapheme.grapheme==g]
    if len(tdf)==0:
        print(g)
        non_found.append(g)
non_found

  0%|          | 0/755 [00:00<?, ?it/s]

র্দ্য
্বা
ব্জা
দ্গু
ৃ
া
ঝিঁ
স্থ্যা
হ্ণে
থ্রী
ষ্ঠূ
হ্যাঁ
চঁ
বূ
্ল
ু
ন্দ্যা
জ্ঞি
ক্ষ্মা
ষ্কি
র্থ্যে
জোঁ
ূ
শঁ
ঁ
ত্র্যে
ন্রা
ে


['র্দ্য',
 '্বা',
 'ব্জা',
 'দ্গু',
 'ৃ',
 'া',
 'ঝিঁ',
 'স্থ্যা',
 'হ্ণে',
 'থ্রী',
 'ষ্ঠূ',
 'হ্যাঁ',
 'চঁ',
 'বূ',
 '্ল',
 'ু',
 'ন্দ্যা',
 'জ্ঞি',
 'ক্ষ্মা',
 'ষ্কি',
 'র্থ্যে',
 'জোঁ',
 'ূ',
 'শঁ',
 'ঁ',
 'ত্র্যে',
 'ন্রা',
 'ে']

In [8]:
def check_not_found(x):
    d=list(set(non_found) & set(x))
    if len(d)>0:
        return d
    else:
        return np.nan
df["not_found"]=df.graphemes.progress_apply(lambda x: check_not_found(x))    

  0%|          | 0/5084 [00:00<?, ?it/s]

In [9]:
df

,words,graphemes,not_found
0,আসবো,"[আ, স, বো]",NaN
1,হিসাবে,"[হি, সা, বে]",NaN
2,বসন্ত,"[ব, স, ন্ত]",NaN
3,মৃত্যুশয্যার,"[মৃ, ত্যু, শ, য্যা, র]",NaN
4,সমদ্র,"[স, ম, দ্র]",NaN
...,...,...,...
5079,বাহিনী,"[বা, হি, নী]",NaN
5080,শয়তান,"[শ, য়, তা, ন]",NaN
5081,মাইক্রো,"[মা, ই, ক্রো]",NaN
5082,দেবীর,"[দে, বী, র]",NaN


In [10]:
df.dropna(inplace=True)

In [11]:
df

,words,graphemes,not_found
5,সা্বাধীনতার,"[সা, ্বা, ধী, ন, তা, র]",[্বা]
41,দুুর্জন,"[দু, ু, র্জ, ন]",[ু]
150,মূূল্যবান,"[মূ, ূ, ল্য, বা, ন]",[ূ]
154,সম্পদ্গুলো,"[স, ম্প, দ্গু, লো]",[দ্গু]
197,যক্ষ্মা,"[য, ক্ষ্মা]",[ক্ষ্মা]
463,দুুর্বল,"[দু, ু, র্ব, ল]",[ু]
483,বাশঁদহ,"[বা, শঁ, দ, হ]",[শঁ]
527,প্রবিন্রা,"[প্র, বি, ন্রা]",[ন্রা]
598,বূলাস,"[বূ, লা, স]",[বূ]
709,পৃৃথিবীতে,"[পৃ, ৃ, থি, বী, তে]",[ৃ]


In [12]:
df.to_csv("not_found.csv",index=False)